In [1]:
count = sum(1 for _ in open("product.csv", "r", encoding="utf-8"))
print(count)

4206882


In [2]:
with open("product.csv", "r", encoding="utf-8") as f:
    first_line = f.readline()

columns = first_line.strip().split(",")
print("Number of columns:", len(columns))
print(first_line)


Number of columns: 1
code	url	creator	created_t	created_datetime	last_modified_t	last_modified_datetime	last_modified_by	last_updated_t	last_updated_datetime	product_name	abbreviated_product_name	generic_name	quantity	packaging	packaging_tags	packaging_en	packaging_text	brands	brands_tags	brands_en	categories	categories_tags	categories_en	origins	origins_tags	origins_en	manufacturing_places	manufacturing_places_tags	labels	labels_tags	labels_en	emb_codes	emb_codes_tags	first_packaging_code_geo	cities	cities_tags	purchase_places	stores	countries	countries_tags	countries_en	ingredients_text	ingredients_tags	ingredients_analysis_tags	allergens	allergens_en	traces	traces_tags	traces_en	serving_size	serving_quantity	no_nutrition_data	additives_n	additives	additives_tags	additives_en	nutriscore_score	nutriscore_grade	nova_group	pnns_groups_1	pnns_groups_2	food_groups	food_groups_tags	food_groups_en	states	states_tags	states_en	brand_owner	environmental_score_score	environmental_score_grade	n

In [4]:
with open("product.csv", "r", encoding="utf-8") as infile, \
     open("sample.csv", "w", encoding="utf-8") as outfile:

    for i in range(20):
        line = infile.readline()
        outfile.write(line)


In [1]:
import pandas as pd

In [8]:
columns = [
    "ingredients_text",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "additives",
    "additives_tags",
    "additives_en",
    "additives_n",
    "allergens",
    "allergens_en",
    "nova_group",
    "nutriscore_score",
    "nutriscore_grade",
    "serving_size",
    "serving_quantity",
    "energy-kcal_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "fiber_100g",
    "salt_100g",
    "sodium_100g"
]

columns2 = [
    "ingredients_text",
    "nutriscore_score",
    "nutriscore_grade",
]

mapping = {
    "a": "Healthy",
    "b": "Healthy",
    "c": "Medium",
    "d": "Not Healthy",
    "e": "Not Healthy"
}



In [17]:
df = pd.read_csv("product.csv", sep="\t", nrows=10000)
df = df[columns2]
df = df.dropna(how="all")
df = df[df["nutriscore_grade"].isin(["a", "b", "c", "d", "e"])]
df = df[df["ingredients_text"].str.len() > 0]
df["healthy_level"] = df["nutriscore_grade"].map(mapping)
print(df.shape)

(782, 4)


C:\Users\Asus\AppData\Local\Temp\ipykernel_348\3581906414.py:1: DtypeWarning: Columns (11,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("product.csv", sep="\t", nrows=10000)


In [19]:
df.to_csv("sample3.csv", index=False)


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load
df = pd.read_csv("sample3.csv")

# Input & Target
X = df["ingredients_text"]
y = df["healthy_level"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text → TF-IDF vectors
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Classifier
model = LogisticRegression(max_iter=300)
model.fit(X_train_vec, y_train)

# Evaluation
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

     Healthy       0.64      0.63      0.64        43
      Medium       0.56      0.19      0.29        26
 Not Healthy       0.73      0.88      0.79        88

    accuracy                           0.69       157
   macro avg       0.64      0.57      0.57       157
weighted avg       0.68      0.69      0.67       157



In [35]:
print(X_test.iloc[0])

Dried cherries, sugar, sunflower oil.


In [36]:
for i in range(25):
    result = model.predict(X_test_vec[i])
    print(f"{X_test.iloc[i]}\t: {result}")

Dried cherries, sugar, sunflower oil.	: ['Not Healthy']
Spanish manzanilla olives, water, pimiento, salt, lactic acid, sodium alginate, guar gum, calcium chloride, potassium sorbate as a preservative.	: ['Not Healthy']
Protein comples (pea protein concentrate, brown rice protein concentrate), Cocoa powder, Medium-chain triglyceride oil powder, Sugars (fructose), Natural flavours, Xanthan gum, Sea salt, Stevia (stevia rebaudiana) leaf extract, Cinnamon.	: ['Not Healthy']
SEMI-SWEET CHOCOLATE: (SUGAR, CHOCOLATE PROCESSED WITH ALKALI, COCOA BUTTER, MILK FAT, SOY LECITHIN-AN EMULSIFIER, VANILLIAN-AN ARTIFICIAL FLAVOR, NATURAL FLAVORS).	: ['Not Healthy']
Fructose, Cellulose Powder, Com Bran 2 scoops (25g) Guar Gum, Calcium Per serving with 2 sps 2g of POM Vanilla 170 200 5% Phosphate, Potassium Chloride, Magnesium Oxide, Calcium Natural and Artifical Cajeta Flavor, Rice Fiber, Canola Oil, Soy Lecithin, Caramel Color 4% Carrageenan Mediu Chain Tynde ule, Sicon Dioxide Ascorbic Acid, Aph Toco

In [71]:
df["ingredients_text"] = df["ingredients_text"].astype(str)

X = df["ingredients_text"]
y = df["healthy_level"]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train2)
y_test = le.transform(y_test2)



In [72]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# 1. Text vectorizer
vectorizer = TextVectorization(
    max_tokens=20000,
    output_sequence_length=200,
)

# vectorizer.adapt(df["ingredients_text"])
vectorizer.adapt(X_train2)

# 2. Build model
model = tf.keras.Sequential([
    vectorizer,
    tf.keras.layers.Embedding(input_dim=20000, output_dim=64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   # 3 classes
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_10           │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_9      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [73]:
# model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
# print(X_train)
# model.fit(X_train2, y_train2, epochs=10, batch_size=32)
y_train2.dtype

dtype('O')

In [ ]:

# Train
model.fit(X_train2, y_train, epochs=10, batch_size=32, validation_split=0.2)


ValueError: Invalid dtype: object

: 